# Notebook - Gold
Notebook responsável por gerar a camada Gold do processo ETL.

A camada Gold consiste em armazenar os dados prontos para análises, ou seja, com agregações e/ou para geração de KPIs. 

No caso:
- Os dados prontos para análises de Ticket médio estão disponíveis na tabela **`gold.order_metrics`**
- Os dados prontos para análises de Taxa de retenção, Número médio de pedidos e Receita média estão disponíveis na tabela **`gold.consumer_metrics`**
- Os dados prontos para análises de Segmentação estão disponíveis na tabela **`gold.consumer_segmented_metrics`**

In [0]:
# Import das bibliotecas
from pyspark.sql.functions import when, col, lit

In [0]:
# Definição das variáveis
catalog = "workspace"
schema_gold = "gold"

path_gold = f"{catalog}.{schema_gold}"

In [0]:
# Criação da tabela gold.order_metrics (Ticket médio)
df_order_metrics = spark.sql("""
    select
        ab.is_target
        , o.delivery_address_state
        , r.price_range
        , o.order_total_amount
    from silver.order o
    inner join silver.ab_test_ref ab on o.customer_id = ab.customer_id
    inner join silver.restaurant r on o.merchant_id = r.id
""")

df_order_metrics.write.mode('overwrite').saveAsTable(f'{path_gold}.order_metrics')

In [0]:
# Criação da tabela gold.consumer_metrics (Taxa de retenção, Número médio de pedidos e Receita média)
df_consumer_metrics = spark.sql("""
    select
        ab.is_target
        , ab.customer_id
        , count(o.order_id) as n_order
        , sum(o.order_total_amount) as sum_amount
    from silver.order o
    inner join silver.ab_test_ref ab on o.customer_id = ab.customer_id
    group by ab.is_target, ab.customer_id
""")

df_consumer_metrics = df_consumer_metrics.withColumn('is_retained', when(col('n_order') > 1, lit(1)).otherwise(lit(0)))

df_consumer_metrics.write.mode('overwrite').partitionBy('is_target').saveAsTable(f'{path_gold}.consumer_metrics')

In [0]:
# Criação da tabela gold.consumer_segmented_metrics (Segmentação)
df_consumer_segmented_metrics = spark.sql("""
    select
        ab.is_target
        , o.delivery_address_state
        , o.origin_platform
        , r.price_range
        , ab.customer_id
        , count(o.order_id) as n_order
        , sum(o.order_total_amount) as sum_amount
    from silver.order o
    inner join silver.ab_test_ref ab on o.customer_id = ab.customer_id
    inner join silver.restaurant r on o.merchant_id = r.id
    group by ab.is_target, o.delivery_address_state, o.origin_platform, r.price_range, ab.customer_id
""")

df_consumer_segmented_metrics = df_consumer_segmented_metrics.withColumn('is_retained', when(col('n_order') > 1, lit(1)).otherwise(lit(0)))

df_consumer_segmented_metrics.write.mode('overwrite').saveAsTable(f'{path_gold}.consumer_segmented_metrics')